In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import sklearn
from sklearn.preprocessing import RobustScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

%matplotlib inline

print("Modules imported successfully!")


# Load the data from the uploaded Excel file
file_path = './VSM_BRIMS_02.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to understand its structure
data.head()

In [ ]:
# Plotting the decision type changes across 15 trials for two example participants (0 and 1)
fig, ax = plt.subplots(figsize=(10, 6))

# Select data for participants 0 and 1
for participant in [0, 1]:
    participant_data = data[data['participant'] == participant]
    ax.plot(participant_data['trial'], participant_data['decision_type'], marker='o', linestyle='-', label=f'participant {participant}')

# Setting plot attributes
ax.set_xlabel('Trial')
ax.set_ylabel('Decision Type')
ax.set_title('Decision Type Changes Across Trials for Participants 0 and 1')
ax.set_yticks([0, 1, 2])
ax.set_yticklabels(['Novice', 'Intermediate', 'Expert'])
ax.legend()

# Show the plot
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

# Convert columns to appropriate types
data['participant'] = data['participant'].astype('category')
data['trial'] = data['trial'].astype(int)
data['decision_type'] = data['decision_type'].astype(int)

# Fit a mixed-effects logistic regression model
model = smf.mixedlm("decision_type ~ trial", data, groups=data["participant"])
result = model.fit()

# Print the summary of the model
print(result.summary())


In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel

data['decision_type'] = data['decision_type'].fillna(0).astype(int)

# Ordered logistic regression model without adding a constant
model = OrderedModel(data['decision_type'], 
                     data[['trial']], 
                     distr='logit')

result = model.fit(method='bfgs', disp=False)

print(result.summary())


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

data = {
    'trial': np.arange(17),
    'mean_decision_type': [0.0, 0.25, 0.5, 0.75, 0.75, 1.0, 1.25, 1.0, 1.25, 1.5, 1.5, 1.75, 1.75, 1.75, 1.75, 1.75, 2.0]
}
df = pd.DataFrame(data)

# Fit a linear regression model
X = df['trial'].values.reshape(-1, 1)
y = df['mean_decision_type'].values
model = LinearRegression()
model.fit(X, y)

# Calculate the learning rate (slope)
learning_rate = model.coef_[0]

# Predict values to calculate residuals
y_pred = model.predict(X)

# Calculate the variance of the residuals
residuals = y - y_pred
variance = np.var(residuals)

# Print the learning rate and variance
print(f"Learning Rate: {learning_rate}")
print(f"Variance of Residuals: {variance}")

# Plot the regression line on the graph for visualization
plt.figure(figsize=(10, 6))
plt.plot(df['trial'], df['mean_decision_type'], marker='o', label='Mean Decision Type')
plt.plot(df['trial'], y_pred, linestyle='--', color='red', label='Linear Fit')
plt.fill_between(df['trial'], df['mean_decision_type'] - np.sqrt(variance), df['mean_decision_type'] + np.sqrt(variance), alpha=0.2)
plt.xlabel('Trial')
plt.ylabel('Decision Type')
plt.title('Overall Trend of Decision Types over Trials with Linear Fit')
plt.legend()
plt.show()
